This file is used to create the vocabulary used in vectorize_final.

In [1]:
import os
import json
import numpy as np
import pandas as pd
import pyarrow as pa
from pandas.io.json import json_normalize
import cryptography
import re
from cryptography.fernet import Fernet
import bs4
from nltk.tokenize import word_tokenize
import pickle
import random

In [4]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)    

regex = re.compile( u'[？！｡。＂＃％＆＇¿（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃《》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏]', re.U)

#ave_obj(regex, 'regex')
#egex = load_obj('regex
def tokenize(text):
    text = text.lower()
    text = re.sub('&quot;','\'',text)
    text = re.sub('&amp;',' ',text)
    text = re.sub(regex, ' ', text)
    text = re.sub('"', '', text)
    #text = re.sub('-', ' ', text)
    text = re.sub(r'\'(?!\')(.*)\'',r'\0', text)
    text = re.sub(r'%\S|\s%|\D%',' ', text)  
    #text = re.sub(r'[,;{}\(\)<>\/�\?\!_\^\|]', ' ', text)
    text = re.sub(r'[\]\[\s,:;{}\(\)<>\/�\?\!_\^\|\*\+\-=.&]', ' ', text)
    
    toks = re.split(r'\s',text)
    toks = [re.sub(r'((?![=]).*)=', r'\0', x) for x in toks]
    #toks = [re.sub(r'((?![.]).*)\.', r'\0', x) for x in toks]
    #toks = [re.sub(r'(^\.(?![.]).*)', r'\0', x) for x in toks]
    toks = [re.sub(r'\\','', x, re.U) for x in toks]   
    toks = [re.sub(r'((?![\]\[]).*)\]', r'\0', x) for x in toks]
    return sorted(toks)


In [2]:
#use to get randomish sample from very large folders 
inds = random.sample(range(0,1000,2), k = 300)

In [6]:
#load data in and use it to construct a vocabularly where the tokens arwe stored with a count of how many documents they 
#have been present in so far
#This currently filters based on url so the whole process doesn't have to run on URLs that have already been seen

# Some dates are missing because they were missing the files I was trying to get
date_batches = [['newbenigndata','2019-04-22','2019-04-23','2019-04-24','2019-04-25','2019-04-26','2019-04-27','2019-04-28','2019-04-29'],
                ['2019-04-09','2019-04-08','2019-04-07'],
                ['2019-04-30','2019-05-01','2019-05-02','2019-05-03','2019-05-04'],
                ['2019-05-05','2019-05-06','2019-05-07','2019-05-08','2019-05-09','2019-05-10','2019-05-11','2019-05-12'],
                ['2019-04-04','2019-04-03','2019-04-02','2019-04-01','2019-05-19','2019-05-20', '2019-04-21','2019-04-20','2019-04-19','2019-04-18','2019-04-17'],
                ['2019-05-21','2019-05-22','2019-05-23','2019-05-24','2019-05-25','2019-05-26','2019-05-28'],
                ['2019-05-29','2019-05-30','2019-05-31','2019-06-01','2019-06-02','2019-06-03','2019-06-04','2019-06-05'],
                ['2019-04-12','2019-04-11','2019-04-10','newNONTORbenigndata'],
                ['2019-06-06','2019-06-07','2019-06-08','2019-06-09','2019-06-10','2019-06-11','2019-06-12','2019-06-13']]
#note that this bottom row is not included in here anymore
dates = date_batches[7]+date_batches[6]+date_batches[4]+date_batches[5]+date_batches[0]+date_batches[2]+date_batches[3]+date_batches[1]
#'2019-04-12','2019-04-16','2019-04-17','2019-04-18','2019-04-19','2019-04-20','2019-04-21]
word_dict = {}
indicators = []
urls = [] 
for date in dates:
    files = os.listdir('/data/data/'+date+'/enrichment/fetch_page')
    top = len(files)
    bot = 0
    if top > 600:
        lst = inds
    else:
        lst = range(bot,top)
    for j in lst:
        with open('/data/data/'+date+'/enrichment/fetch_page/'+files[j]) as d:
            r = json.load(d)
            try:
                df = json_normalize(r['data'])
            except:
                break
        try:
            df = df[df['status_code'] == 200].reset_index(drop=True)
            df = df[df['success']]
        except:
            df = df[df['success']]
        #not worrying about labels right now
        html_str = df['page_content'].copy()
        
        for i in range(len(html_str)):
            if df.loc[i,'url'] in urls:
                continue
            else:
                urls.append(df.loc[i, 'url'])
            key = bytes( df.loc[i,'encryption_key'],encoding = 'UTF-8')
            f = Fernet(key) #all rows of this df share this key
            #decrypt string
            text = f.decrypt(bytes(html_str[i],encoding='UTF-8'))
            #clean out symbols
            text = text.decode('utf8')
            toks = tokenize(text)
           # print(toks[10:15])
            toks = set(toks)
            for t in toks:
                if t != '' and t != '\x00' and len(t) > 1:
                    if t in word_dict.keys():
                        word_dict[t] = word_dict.get(t) + 1
                        #print(t)
                    else:
                        word_dict[t] = 1
                        #if '%' in t:
                        #print(t)

            #add to set
        print(len(word_dict))
         #end of for
    #end of for
    print('------------'+date+'---------------')
#end of for


           
            

12714914
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715325
12715388
12715388
12715984
12715987
12715987
12715987
12715987
12715987
12715987
12715987
12715987
12715987
12716113
12716113
12716113
12716113
12716164
12716164
12716164
12716164
12716165
12716165
12716165
12716165
12716165
12716165
12716167
12716167
12716167
12716167
------------2019-04-12---------------
12716167
12716167
12716167
12716167
12716167
12716167
12716167
12716167
12716167
12716167
12716167
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716303
12716447
12716447
12716447
12716447
12716447
12716447
12716447
------------2019-04-11---------------
12716447
12716468
12716468
12716468
12716595
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
12716600
127166

KeyboardInterrupt: 

In [7]:
#save the needed files
save_obj(word_dict, 'plain_word_dict')
pairs = sorted(word_dict.items(), key=lambda x: x[1], reverse=True)
save_obj(pairs, 'plain_sorted_kvpairs')